<h1><span style="color:red">Converting a #multi variable into a series of binary variables</span></h1>

Using this notbook, you can select a multiple-response variable and convert it into a series of binary variables, with values "Selected" and empty, possibly dropping the initial binary variables. You will have an option to process a survey file received from the current SuAVE application, or import a local CSV file. Then the notebook will let you create a new SuAVE survey with the updated survey file. 

The binary variables will be named as values in the original #multi variables.


## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

## 2. Import libraries

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
import pandas as pd
from IPython.display import Markdown, display
import pandas as pd
import panel as pn
import numpy as np

pn.extension()

def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"
url_partitioned = full_notebook_url.partition('/operations')
base_url = url_partitioned[0];

In [ ]:
def slider(df):
    """
    slider creates an interactive display of a
    data frame.
    
    :param df: data frame
    :returns: interactive dataframe
    """
    
    # Row Selector widget
    row_selection = pn.widgets.IntSlider(name='Navigate Rows', width=350, 
                                         margin=(0,50,-15,0), end=len(df)-1)

    # Column Selector widget
    col_selection = pn.widgets.IntSlider(name='Navigate Columns', width=350, 
                                         margin=(0,0,5,0), end=len(df.columns))
    
    @pn.depends(row_selection.param.value, col_selection.param.value)
    def navigate_data(row=0, col=0):
        return df.iloc[row:row+5, col:col+10]
    
    sliders = pn.Row(row_selection, col_selection, margin=(0,0,0,10))
    full_widget = pn.Column(sliders, navigate_data)
    return full_widget

# reading a data file. Replace with survey file retrieved from SuAVE 

def extract_data(path):
    """
    extract_data reads files from various formats
    
    :param link: string representing path to file
    :returns: data frame of file
    """

    # Reading file at path
    if path.endswith(('.txt', 'tsv')):
        try:
            data = pd.read_csv(path, sep='\t', encoding="latin-1")
        except UnicodeDecodeError:
            data = pd.read_csv(path, sep='\t', encoding="ISO-8859-1")
    elif path.endswith('.csv'):
        try:
            data = pd.read_csv(path, encoding="latin-1")
        except UnicodeDecodeError:
            data = pd.read_csv(path, encoding="ISO-8859-1")
    else:
        return None
    
    return data


In [ ]:
# this function will create a set of binary variables given one #multi variable

def binarize(df, column, true_value=1, false_value=0, separator='|'):
    """
    return dataframe with additional new columns that binarize the values of this column
    parameters:
        * df: original dataframe
        * column: name of column that has multiple values per cell
        * separator: how the values are separated in the original column, e.g. by commas
        * true_value: what value dummy variables will take if present, e.g. 1 or 'Yes'
        * false_value: what value dummy variables will take if absent, e.g. 0 or 'No' or np.nan
    returns merged dataframe with new columns
    """
    dummies = df[column].str.get_dummies(sep=separator)
    for col in dummies.columns:
        dummies[col] = dummies[col].map({1: true_value, 0: false_value})
        print("Creating binary variable : '"+ col + "' from multiple-response variable : '" + column +"'")

    merged = pd.merge(df, dummies, left_index=True, right_index=True, how='inner')
    assert len(merged) == len(df) == len(dummies)
    return merged

# this function calls binarize for all defined #multi variables

def binarize_list(df, multi_columns, true_value=1, false_value=0, separator='|'):
    for col in multi_columns:
        df = binarize(df, col, true_value=true_value, false_value=false_value, separator=separator)
    return df

# this function deletes the #multi variables after processing

def delete_multies(df, list_columns):
    for col in list_columns:
        df = df.drop(col, axis=1)
    return df


## 3. Select a survey file from SuAVE or import a local CSV file

In [ ]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file'], 
                                       inline=False)
data_select

In [ ]:
data_input = pn.widgets.FileInput()
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red'>SuAVE survey will be loaded. Continue to step 4.</span></b>")

    else:
        message = pn.pane.HTML("<b><span style='color:red'>Upload data and continue to step 4.</span></b>")
        return pn.Column(message, data_input)
    
check_selection()

## 4. Visualize the data and select #multi variables to process

In [ ]:
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)

# df = extract_data(fname).fillna('')
df = extract_data(fname)

slider(df)

In [ ]:
# Multi column selector
left_text = pn.Row("####Select #multi Variables to Process", margin=(0,0,-15,210))
multi_options = list(df.columns[['#multi' in col for col in df.columns]])
multi_select = pn.widgets.CrossSelector(height = 130, options=multi_options)

multi_cols = []
def save_multi(click):
    global multi_cols
    multi_cols = multi_select.value
    
# Process button
process_button = pn.widgets.Button(name='End Selection', button_type='primary', width=200, margin=(10,0,10,240))
process_button.param.watch(save_multi, ['clicks'])

# Display widgets
full_display = pn.Column(left_text, multi_select, process_button, css_classes=['widget-box'])
full_display

## 5. Generate binary variables from #multi, and optionally remove the #multi

In [ ]:
# run to create a new df with added binary variables

df_new = binarize_list(df, 
         multi_cols,
         true_value='Selected',
         false_value=np.nan,
         separator='|')

In [ ]:
# delete the original #multi vars if needed
df_new = delete_multies(df_new,multi_cols)
df = df_new.copy().fillna('')

In [ ]:
slider(df)

In [ ]:
# save as new file (eventually, upload to SuAVE)
df.to_csv('test_2binary.csv', index=None)

## 6. Generate a new survey and open it in SuAVE

In [ ]:
# new filename

if data_select.value == 'Import a local CSV file':
    csv_file = data_input.filename

new_file = absolutePath + csv_file[:-4]+'_v1.csv'
printmd("<b><span style='color:red'>A new temporary file will be created at: </span></b>")
print(new_file)
df.to_csv(new_file, index=None)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

In [ ]:
referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file, "rb")}

if data_select.value == 'Import a local CSV file':
    dzc_file = ''
    views = '1110001'
    view='grid'

upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + s_url + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)